back-translate하는 모델을 선정하는데 많은 시간을 들였으나 그럼에도 눈으로 보기에 매끄럽지 않은 부분이 있었음
수동적으로 데이터를 걸러내는 작업이 필수적으로 필요할것으로 보임

In [5]:
from tqdm import tqdm
from transformers import MarianTokenizer, MarianMTModel, M2M100ForConditionalGeneration, AutoTokenizer
import torch
import pandas as pd
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df_train = pd.read_csv('data/train.csv')
df_general = pd.read_csv('line_general.csv', sep=',', encoding='utf-8')

print("Train data shape:", len(df_train))
print(df_train.head())

print("General data shape:", len(df_general))
print(df_general.head())

df = pd.concat([df_train, df_general], ignore_index=True)
print("Merged data shape:", len(df))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

src_t = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ko-en')
src_m = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-ko-en').to(device)

tgt_t = AutoTokenizer.from_pretrained('NHNDQ/nllb-finetuned-en2ko')
tgt_m = M2M100ForConditionalGeneration.from_pretrained('NHNDQ/nllb-finetuned-en2ko').to(device)


@torch.no_grad()
def back_translate(text):
    # ko → en
    ko_token = src_t(text, return_tensors="pt", truncation=True).to(device)
    en_ids = src_m.generate(**ko_token, num_beams=5)
    en_text = src_t.decode(en_ids[0], skip_special_tokens=True)

    # en → ko
    en_token = tgt_t(en_text, return_tensors="pt", truncation=True).to(device)
    ko_ids = tgt_m.generate(**en_token, num_beams=5)
    ko_text = tgt_t.decode(ko_ids[0], skip_special_tokens=True)

    return en_text, ko_text


def split_nonempty_lines(text):
    return [ln.strip() for ln in str(text).split("\n") if ln.strip()]


# ✅ conversation 하나를 처리하는 함수 (배치 없음)
def back_translate_conversation(text):
    lines = split_nonempty_lines(text)

    en_list = []
    ko_list = []

    for line in lines:
        en_text, ko_text = back_translate(line)
        en_list.append(en_text)
        ko_list.append(ko_text)

    # ko_list를 다시 하나의 conv로 합침
    ko_conv = "\n".join(ko_list)

    return en_list, ko_list, ko_conv


# ✅ 전체 데이터 처리
new_rows = []
for i in tqdm(range(len(df)), desc="Back Translating"):
    original_row = df.iloc[i]

    en_list, ko_list, ko_conv = back_translate_conversation(original_row["conversation"])

    new_rows.append({
        "idx": original_row["idx"],       # 원본 유지
        "class": original_row["class"],   # 원본 유지
        "conversation": ko_conv,          # 변환된 한국어 대화
    })

    if i % 10 == 0:
        print(f"Original Conversation (idx={original_row['idx']}):\n{original_row['conversation']}\n")
        print(f"Back Translated Conversation:\n{ko_conv}\n")
        print("-" * 50)

new_df = pd.DataFrame(new_rows)


Train data shape: 3950
   idx      class                                       conversation
0    0      협박 대화  지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1    1      협박 대화  길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2    2  기타 괴롭힘 대화  너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3    3      갈취 대화  어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4    4      갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
General data shape: 800
   idx class                                       conversation
0    0    일반  오늘 날씨 진짜 좋다.\n그치?\n하늘도 맑고.\n응, 어디 놀러 가고 싶어지네.\...
1    1    일반  점심 뭐 먹을래?\n글쎄.\n어제 치킨 먹어서 오늘은 좀 담백한 거.\n그럼 파스타...
2    2    일반  요즘 뭐 재미있는 드라마 있어?\n아, 나 새로 시작한 거 있는데 완전 내 스타일이...
3    3    일반  주말에 뭐 했어?\n친구 만나서 영화 봤어.\n너는?\n나는 집에서 뒹굴뒹굴.\nㅋ...
4    4    일반  퇴근하고 뭐 해?\n바로 집에 가지.\n피곤해서.\n그치, 나도.\n집에 가서 저녁...
Merged data shape: 4750


c:\Users\tkdwl\miniconda3\envs\aiffel\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Back Translating:   0%|          | 1/4750 [00:02<3:15:03,  2.46s/it]

Original Conversation (idx=0):
지금 너 스스로를 죽여달라고 애원하는 것인가?
 아닙니다. 죄송합니다.
 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.
 정말 잘못했습니다.
 너가 선택해. 너가 죽을래 네 가족을 죽여줄까.
 죄송합니다. 정말 잘못했습니다.
 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.
 선택 못하겠습니다. 한번만 도와주세요.
 그냥 다 죽여버려야겠군. 이의 없지?
 제발 도와주세요.

Back Translated Conversation:
내가 죽여줄까?
아니, 미안해.
죽으실 거면 우리 혼자 죽게 해주시고, 우리가 사건에 관여하게 해주신다고요?
진짜 틀렸어.
네가 선택해 내가 죽고 싶다 내가 죽고 싶다 네가 가족을 죽이고 싶다.
죄송합니다, 정말 죄송합니다.
선택의 여지가 없으세요, 선택하지 않으면 당신과 당신의 가족을 죽여버리겠습니다.
저는 선택의 여지가 없습니다. 당신이 저를 도와야 합니다.
그냥 다 죽여야지, 동의하지 않아?
좀 도와주세요.

--------------------------------------------------


Back Translating:   0%|          | 11/4750 [00:30<3:50:20,  2.92s/it]

Original Conversation (idx=10):
네 뭐 드릴까요?
말보르 레드 하나.
네. 여기요!
여기 사장은 없어?
네 사장님 지금 안계세요. 뭐 .찾으세요?
지금부터 내 말 잘들어. 여기 주머니에 있는 돈 다 담아.
 네?? 왜 이러세요!!
 입닥쳐. 빨리 사람 오기전에 돈이나 쓸어담어.
 .살려만 주세요 제발
 제대로 안하면 패버릴거니까 빨리빨리 해라.
네.여기요.
진작 이렇게 빨리 줄것이지. 경찰 신고하면 알지?

Back Translated Conversation:
무엇을 드릴까요?
원 말보 레드.
네, 여기요!
여기 사장님 없으신가요?
네, 지금 미스터 그랜트가 안 계세요. 무엇을 찾고 계신가요?
이제부터 내 말 들어봐 이 주머니에 돈 다 넣어.
뭐 하는 거야?
shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shut shu

...

Back Translating: 100%|█████████▉| 4731/4750 [5:13:01<00:20,  1.08s/it]

Original Conversation (idx=780):
요즘 배달 앱 할인 많더라.
맞아 자주 확인해.
뭐 시켰어?
치킨.
할인 많이 받았어?
꽤 됐어.
부럽다.

Back Translated Conversation:
요즘 배송 할인이 많이 들어오고 있습니다.
네, 많이 확인해 보세요.
뭐 했어?
치킨.
할인을 많이 받으셨나요?
오랜만이다.
부럽다.

--------------------------------------------------


Back Translating: 100%|█████████▉| 4741/4750 [5:13:11<00:08,  1.00it/s]

Original Conversation (idx=790):
요즘 스트레스 푸는 방법 있어?
산책하거나 음악 들어.
도움 돼?
응 기분이 나아져.
나도 해봐야겠다.
추천해.

Back Translated Conversation:
요즘 스트레스를 받을 수 있는 방법이 있나요?
걷거나 음악을 듣습니다.
도와주실 수 있나요?
그래, 기분이 좋아졌어.
나도 할 거야.
추천합니다.

--------------------------------------------------


Back Translating: 100%|██████████| 4750/4750 [5:13:21<00:00,  3.96s/it]


In [6]:
new_df.to_csv('data/back_translated_data.csv', index=False)